In [ ]:
!unzip trainingData.zip

# Some basic Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, json, cv2, numpy as np, matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F
import albumentations as A 

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/MaskKeypointRCNN-pytorch-master')

In [ ]:
import transforms, utils, engine, train
from utils import collate_fn
from engine import train_one_epoch, evaluate

## Augmentations if necessary

In [ ]:
# Augmentations
def train_transform():
    return A.Compose([
        A.Sequential([
            A.RandomRotate90(p=0), # Random rotation of an image by 90 degrees zero or more times
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, brightness_by_max=True, always_apply=False, p=1), # Random change of brightness & contrast
        ], p=1)
    ],
    keypoint_params=A.KeypointParams(format='xy'), # More about keypoint formats used in albumentations library read at https://albumentations.ai/docs/getting_started/keypoints_augmentation/
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['bboxes_labels']) # Bboxes should have labels, read more at https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    )

## Dataset class

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
import random
import os
import cv2
from matplotlib import pyplot as plt
%matplotlib inline



class MaskKeypointDataset(Dataset):
    def __init__(self, img_dir, lab_path, transforms,
                 keypoint_names=("mesial", "distal", "apical"),
                 vis=False):
        self.vis = vis

        self.img_dir = img_dir
        self.transforms = transforms
        self.keypoint_names = keypoint_names
        self.num_keypoints = len(keypoint_names)

        self.img_fnames = []
        self.scatter_keypointss = []
        self.mask_polygonss = []
        with open(lab_path, "r") as f:
            labels = json.load(f)
            labels = list(labels.values())
        for lab in labels:
            crop_fname = lab["filename"]
            mask_polygons = [list(zip(reg["shape_attributes"]["all_points_x"], reg["shape_attributes"]["all_points_y"]))
                             for reg in lab["regions"] if reg["shape_attributes"]["name"] == "polygon"]

            scatter_keypoints = [(reg["region_attributes"]["keypoint"], reg["shape_attributes"]["cx"], reg["shape_attributes"]["cy"])
                                 for reg in lab["regions"] if reg["shape_attributes"]["name"] == "point"]  # (kp_name, x, y)

            if (len(scatter_keypoints) > 0):
                self.img_fnames.append(crop_fname)
                #print(self.img_fnames)
                self.scatter_keypointss.append(scatter_keypoints)
                #print(self.scatter_keypointss)
                self.mask_polygonss.append(mask_polygons)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_fnames[idx])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        polygons = self.mask_polygonss[idx]
        scatter_kps = self.scatter_keypointss[idx]
        bboxes = []
        masks = []
        keypointss = []
        for polygon in polygons:
            bbox = (min([p[0] for p in polygon]), min([p[1] for p in polygon]),
                    max([p[0] for p in polygon]), max([p[1] for p in polygon]))  # (xmin, ymin, xmax, ymax)
            bboxes.append(bbox)

            mask = cv2.fillPoly(img=np.zeros_like(image[:, :, 0]), pts=[np.expand_dims(np.array(polygon), axis=1)], color=(1,))
            masks.append(mask)

            keypoints = []
            for kp_name in self.keypoint_names:
                candidates = [kp for kp in scatter_kps if kp[0] == kp_name]
                #print(candidates)
                chosen = None
                for kp in candidates:
                    #print(mask[kp[2], kp[1]])
                    if mask[kp[2], kp[1]] == 1:
                        chosen = (kp[1], kp[2], 1)  # (x, y, visibility)
                        break
                if chosen is None:
                    chosen = [0, 0, 0]
                keypoints.append(chosen)
            keypointss.append(keypoints)
            #print(keypointss)
        bboxes = torch.as_tensor(bboxes, dtype=torch.float32)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        keypointss = torch.as_tensor(keypointss, dtype=torch.float32)

        image_id = torch.tensor([idx])
        labels = torch.ones((len(bboxes),), dtype=torch.int64)  # only have one class
        iscrowd = torch.zeros((len(bboxes),), dtype=torch.int64)  # all masks are not crowd
        #area = (bboxes[:, 3] - bboxes[:, 1]) * (bboxes[:, 2] - bboxes[:, 0])
        area = torch.tensor([0,0,0,0])
        if len(bboxes>0):
          area = (bboxes[:, 3] - bboxes[:, 1]) * (bboxes[:, 2] - bboxes[:, 0])
        target = {}
        target["image_id"] = image_id
        target["boxes"] = bboxes
        target["keypoints"] = keypointss
        target["masks"] = masks
        target["labels"] = labels
        target["iscrowd"] = iscrowd
        target["area"] = area
        if self.transforms is not None:
            image, target = self.transforms(image, target)

        if self.vis:
            self.visualize(image, target)

        return image, target

    def __len__(self):
        return len(self.img_fnames)

    @ classmethod
    def collate_fn(cls, batch):
        return tuple(zip(*batch))  # Default collate_fn needs same size of images, so should re-define it here

    def visualize(self, image, target):
        image_cv2 = cv2.cvtColor((image.permute((1, 2, 0)).numpy() * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
        for bbox, keypoints, mask in zip(target["boxes"], target["keypoints"], target["masks"]):
            xmin, ymin, xmax, ymax = map(int, bbox)
            cv2.rectangle(image_cv2, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            image_cv2[:, :, 0] = np.where(mask.numpy() == 1, (255 + image_cv2[:, :, 0]) // 2, image_cv2[:, :, 0])
            for i, kp in enumerate(keypoints):
                x, y, visibility = map(int, kp)
                if visibility == 1:
                    cv2.circle(image_cv2, (x, y), 2, (0, 0, 255), 2)
                    cv2.putText(image_cv2, self.keypoint_names[i], (x, y-2), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1)
        cv2.imshow("", image_cv2); cv2.waitKey(0)



def get_transforms(train):
    class ToTensor(object):
        def __call__(self, image, target):
            image = image.astype(np.float32) / 255
            image = torch.as_tensor(image).permute((2, 0, 1))
            return image, target

    class RandomHorizontalFlip(object):
        def __init__(self, prob):
            self.prob = prob

        def __call__(self, image, target):
            if random.random() < self.prob:
                height, width = image.shape[-2:]
                image = image.flip(-1)
                target["masks"] = target["masks"].flip(-1)

                bbox = target["boxes"]
                bbox[:, [0, 2]] = width - bbox[:, [2, 0]]
                target["boxes"] = bbox
                keypoints = target["keypoints"]
                for keypoint in keypoints:
                    for kp in keypoint:
                        kp[0] = width - kp[0]
                target["keypoints"] = keypoints
            return image, target

    class Compose(object):
        def __init__(self, transforms):
            self.transforms = transforms

        def __call__(self, image, target):
            for t in self.transforms:
                image, target = t(image, target)
            return image, target

    transforms = []
    transforms.append(ToTensor())
    if train:
        transforms.append(RandomHorizontalFlip(0.5))
    return Compose(transforms)


def get_dataloader(img_dir, lab_path, kp_names,
                   train, batch_size, val_split=0.0, shuffle=True, num_workers=0):
    dataset = MaskKeypointDataset(img_dir=img_dir, lab_path=lab_path, keypoint_names=kp_names, transforms=get_transforms(train=train))
    if val_split == 0.0:
        dataset_train = dataset
        dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=MaskKeypointDataset.collate_fn)
        dataloader_val = None
    else:
        indices = torch.randperm(len(dataset)).tolist()
        where_train = int(len(dataset) * (1 - val_split))
        dataset_train = torch.utils.data.Subset(dataset, indices[:where_train])
        dataset_val = torch.utils.data.Subset(dataset, indices[where_train:])
        dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=MaskKeypointDataset.collate_fn)
        dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=MaskKeypointDataset.collate_fn)
    return dataloader_train, dataloader_val


if __name__ == '__main__':
    dataset = MaskKeypointDataset(img_dir=r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\new\img",
                                  lab_path=r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\tmp_database\data (19).json",
                                  keypoint_names=("apical", "mesial", "distal"),
                                  transforms=get_transforms(train=True),
                                  vis=True)
    it = iter(dataset)
    while True:
        try:
            a = next(it)
        except StopIteration:
            break

In [ ]:
# Visualizing a random item from dataset

In [ ]:
keypoints_classes_ids2names = {0:'apical', 1: 'mesial', 2: 'distal'}
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
iterator = iter(data_loader)
batch = next(iterator)
def visualize(image, bboxes, keypoints, image_original=None, bboxes_original=None, keypoints_original=None, scores=[], keypoints_scores=[]):
    fontsize = 1
    i = 0
    for bbox in bboxes:
        start_point = (bbox[0], bbox[1])
        end_point = (bbox[2], bbox[3])
        image = cv2.rectangle(image.copy(), start_point, end_point, (0,255,0), 2)
        if len(scores) > 0:
          image = cv2.putText(image.copy(), "Detected " + str(int(100 * scores[i]) / 100), (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 2, cv2.LINE_AA)
        else:
          image = cv2.putText(image.copy(), "Detected ", (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 2, cv2.LINE_AA)
        i += 1

    j = 0
    for kps in keypoints:
        for idx, kp in enumerate(kps):
            image = cv2.circle(image.copy(), tuple(kp), 5, (255,0,0), 10)
#           image = cv2.putText(image.copy(), " " + keypoints_classes_ids2names[idx], tuple(kp), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1, cv2.LINE_AA)
            if len(keypoints_scores) > 0:
              image = cv2.putText(image.copy(), " " + keypoints_classes_ids2names[idx] + " " + str(int(100 * keypoints_scores[j][idx]) / 100), tuple(kp), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
            else:
              image = cv2.putText(image.copy(), " " + keypoints_classes_ids2names[idx], tuple(kp), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
        j += 1
    #print(keypoints)
    cv2.line(image, tuple(keypoints[0][0]), tuple(keypoints[0][1]), (0,0,255), thickness=2) 
    cv2.line(image, tuple(keypoints[0][0]), tuple(keypoints[0][2]), (255,0,0), thickness=2)

    if image_original is None and keypoints_original is None:
        plt.figure(figsize=(40,40))
        plt.imshow(image)

    else:
        for bbox in bboxes_original:
            start_point = (bbox[0], bbox[1])
            end_point = (bbox[2], bbox[3])
            image_original = cv2.rectangle(image_original.copy(), start_point, end_point, (0,255,0), 2)
        
        for kps in keypoints_original:
            for idx, kp in enumerate(kps):
                image_original = cv2.circle(image_original, tuple(kp), 5, (255,0,0), 10)
                image_original = cv2.putText(image_original, " " + keypoints_classes_ids2names[idx], tuple(kp), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 3, cv2.LINE_AA)

        f, ax = plt.subplots(1, 2, figsize=(40, 20))

        ax[0].imshow(image_original)
        ax[0].set_title('Original image', fontsize=fontsize)

        ax[1].imshow(image)
        ax[1].set_title('Transformed image', fontsize=fontsize)
        
image = (batch[0][0].permute(1,2,0).numpy() * 255).astype(np.uint8)
bboxes = batch[1][0]['boxes'].detach().cpu().numpy().astype(np.int32).tolist()

keypoints = []
for kps in batch[1][0]['keypoints'].detach().cpu().numpy().astype(np.int32).tolist():
    keypoints.append([kp[:2] for kp in kps])



visualize(image, bboxes, keypoints)

# Model building and training

In [ ]:
def get_model(num_keypoints, weights_path=None):
    
    anchor_generator = AnchorGenerator(sizes=(32, 64, 128, 256, 512), aspect_ratios=(0.25, 0.5, 0.75, 1.0, 2.0, 3.0, 4.0))
    model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=False,
                                                                   pretrained_backbone=True,
                                                                   num_keypoints=num_keypoints,
                                                                   num_classes = 2, # Background is the first class, object is the second class
                                                                   rpn_anchor_generator=anchor_generator)

    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)        
        
    return model

In [ ]:
import math
import sys
import time

import torch
import torchvision.models.detection.mask_rcnn
import utils
from coco_eval import CocoEvaluator
from coco_utils import get_coco_api_from_dataset

from collections import deque 


def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq, scaler=None):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter("lr", utils.SmoothedValue(window_size=1, fmt="{value:.6f}"))
    header = f"Epoch: [{epoch}]"

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )

    max_length = 500
    loss_values = deque([np.inf, np.inf, np.inf, np.inf, np.inf], maxlen = max_length) #It'll contain 5 losses, it has fixed size, everything added will delete the first element
    loss_values_array = []
    for images, targets in metric_logger.log_every(data_loader, print_freq, header):

      
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.cuda.amp.autocast(enabled=scaler is not None):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        if scaler is not None:
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            losses.backward()
            optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

        ## Early stopping 
        
        print(loss_values)
        
        i = 0
        for i in range(max_length):
          if loss_value >= loss_values[i]:
            i += 1
          else:
            break

        if i == max_length:
          print("EARLY STOPPING!")
          break
        
        loss_values.append(loss_value)
        loss_values_array.append(loss_value)

    return metric_logger, loss_values_array

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dataset = MaskKeypointDataset(img_dir=r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\new\img",
                                  lab_path=r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\tmp_database\data (19).json",
                                  keypoint_names=("apical", "mesial", "distal"),
                                  transforms=get_transforms(train=True),
                                  vis=False)
#data_loader_train = DataLoader(train_set, batch_size=1, shuffle=True, collate_fn=collate_fn)
#data_loader_val = DataLoader(val_set, batch_size=1, shuffle=True, collate_fn=collate_fn)
#data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=True, collate_fn=collate_fn)
model = get_model(num_keypoints = 3)
#model.load_state_dict(torch.load('/content/drive/MyDrive/brain loss/keypointsrcnn_weights.pth'))
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0060, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.3)
criterion = torch.nn.CrossEntropyLoss()
num_epochs = 10



## Cross validation
from sklearn.model_selection import KFold 
kf = KFold(n_splits=5)
for fold, (train_idx,val_idx) in enumerate(kf.split(dataset)):
  
  print('------------fold no---------{}----------------------'.format(fold))

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
  test_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)
 

  trainloader = DataLoader(dataset, batch_size=10, collate_fn=collate_fn, sampler=train_subsampler)
  valloader = DataLoader(dataset, batch_size=10, collate_fn=collate_fn, sampler=test_subsampler)
 
 
  for epoch in range(num_epochs):
    _, loss_values_array = train_one_epoch(model, optimizer, trainloader, device, epoch, print_freq=1)
    #print(loss_values_array)
    #plt.scatter(np.arange(len(loss_values_array)),loss_values_array)
    plt.plot(loss_values_array)
    plt.title("Loss")
    plt.show()
    print("-------------EVALUATION : ---------------")
    evaluate(model.to(torch.device('cuda')), valloader, torch.device('cuda'))
    break


# Save model weights after training
torch.save(model.state_dict(), 'keypointsrcnn_weights.pth')


# Hyperparameter tuning

In [ ]:
#Uncomment the following line if you'll first use ray tune
#!pip install -U "ray[tune]"

In [ ]:
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1' 
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


dataset = MaskKeypointDataset(img_dir=r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\new\img",
                                  lab_path=r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\tmp_database\data (19).json",
                                  keypoint_names=("apical", "mesial", "distal"),
                                  transforms=get_transforms(train=True),
                                  vis=False)


############################################## RAY TUNE ####################################################### 
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from filelock import FileLock
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


# Define the config dictionary
config = {
    "lr": tune.loguniform(1e-4, 1e-2),  # Learning rate from 1e-4 to 1e-1
    "batch_size": tune.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]),  # list of batch_size
    "data_dir": dataset,
}



from sklearn.model_selection import KFold 

early_stop = True
n_epochs_stop = 4
epochs_no_improve = 0
num_epochs = 10
scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=num_epochs,
    grace_period=1,
    reduction_factor=2)
reporter = CLIReporter(
    # parameter_columns=["l1", "l2", "lr", "batch_size"],
    metric_columns=["loss", "accuracy", "training_iteration"])
    
def train_model(config):
  model = get_model(num_keypoints = 3)
  device = "cpu"
  if torch.cuda.is_available():
    device = "cuda:0"
    if torch.cuda.device_count() > 1:
      model = nn.DataParallel(model)
  model.to(device)
  params = [p for p in model.parameters() if p.requires_grad]
  optimizer = torch.optim.SGD(params, lr=config["lr"], momentum=0.9, weight_decay=0.0005)
  #lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.3)
  criterion = torch.nn.CrossEntropyLoss()
  ## Cross validation
  kf = KFold(n_splits=5)
  for fold, (train_idx,val_idx) in enumerate(kf.split(dataset)):
    
    print('------------fold no---------{}----------------------'.format(fold))

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    test_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)
  

    trainloader = DataLoader(dataset, batch_size=config["batch_size"], collate_fn=collate_fn, sampler=train_subsampler)
    valloader = DataLoader(dataset, batch_size=config["batch_size"], collate_fn=collate_fn, sampler=test_subsampler)
  
  
    for epoch in range(num_epochs):
      train_one_epoch(model, optimizer, trainloader, device, epoch, print_freq=1)

      print("-------------EVALUATION : ---------------")
      evaluate(model.to(torch.device('cuda')), valloader, torch.device('cuda'))
      break


gpus_per_trial = 1
# ...
result = tune.run(
    partial(train_model),
    resources_per_trial={"cpu": 8, "gpu": gpus_per_trial},
    config=config,
    checkpoint_at_end=True,
    num_samples=2,
    progress_reporter=reporter,
    scheduler=scheduler
    )

# Save model weights after training
#torch.save(model.state_dict(), 'keypointsrcnn_weights.pth')

In [ ]:
torch.save(model.state_dict(), 'keypointsrcnn_weights.pth')

# External Validation 

In [ ]:
from sklearn.model_selection import train_test_split

dataset = MaskKeypointDataset(img_dir=r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\new\untitled_folder",
                                  lab_path=r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\tmp_database\data_val.json",
                                  keypoint_names=("apical", "mesial", "distal"),
                                  transforms=get_transforms(train=True),
                                  vis=False)

#X_train, X_test = torch.utils.data.random_split(dataset, [1074, 461]) 



model = get_model(num_keypoints = 3)
data_loader_val = DataLoader(dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

model.load_state_dict(torch.load(r"C:\Users\Babak\Downloads\keypointsrcnn_weights.pth"))

evaluate(model.to(torch.device('cuda')), data_loader_val, torch.device('cuda'))

In [ ]:
iterator = iter(valloader)
images, targets = next(iterator)
images = list(image.to(device) for image in images)

with torch.no_grad():
    model.to(device)
    model.eval()
    output = model(images)

print("Predictions: \n", output)

In [ ]:
for filename in os.listdir(r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\new\untitled_folder"):
  image = cv2.imread(os.path.join(r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\new\untitled_folder", filename))
  print(filename)
  print(image)


In [ ]:
import os
from PIL import Image

yourpath = r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\new\untitled_folder"
for root, dirs, files in os.walk(yourpath, topdown=False):
    for name in files:
        print(os.path.join(root, name))
        if os.path.splitext(os.path.join(root, name))[1].lower() == ".tif":
            if os.path.isfile(os.path.splitext(os.path.join(root, name))[0] + ".jpg"):
                print ("A jpeg file already exists for %s" % name)
            # If a jpeg is *NOT* present, create one from the tiff.
            else:
                outfile = os.path.splitext(os.path.join(root, name))[0] + ".jpg"
                try:
                    im = Image.open(os.path.join(root, name))
                    print ("Generating jpeg for %s" % name)
                    im.thumbnail(im.size)
                    im.convert("RGB").save(outfile, "JPEG", quality=100)
                except Exception as e:
                    print (e)


In [ ]:
import os

dir_name = r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\new\untitled_folder"
test = os.listdir(dir_name)

for item in test:
    if item.endswith(".tif"):
        os.remove(os.path.join(dir_name, item))

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
for filename in os.listdir(r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\new\untitled_folder"):
  image = cv2.imread(os.path.join(r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\new\untitled_folder", filename))
  model.to(device)
  output = model(images)
  scores = output[0]['scores'].detach().cpu().numpy()
  keypoints_scores = output[0]['keypoints_scores'].detach().cpu().numpy()
  high_scores_idxs = np.where(scores > 0.9)[0].tolist() # Indexes of boxes with scores > 0.7
  post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], output[0]['scores'][high_scores_idxs], 0.5).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)
  keypoints = []
  for kps in output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
    keypoints.append([list(map(int, kp[:2])) for kp in kps])
  bboxes = []
  for bbox in output[0]['boxes'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
      bboxes.append(list(map(int, bbox.tolist())))
visualize(image, bboxes, keypoints, scores = scores, keypoints_scores = keypoints_scores)
print(filename)

In [ ]:
import torchvision
from PIL import Image
image_show = cv2.imread(r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\new\untitled_folder\1.jpg")
image_ = Image.open(r"C:\Users\Babak\Downloads\MaskKeypointRCNN-pytorch-master\data\new\untitled_folder\1.jpg").convert('RGB')

image = torchvision.transforms.ToTensor()(image_)


img = image.permute(0, 2, 1)
img = torch.unsqueeze(img, 0)

model.to(device)
image.to(device)

output = model(img.cuda())

scores = output[0]['scores'].detach().cpu().numpy()
keypoints_scores = output[0]['keypoints_scores'].detach().cpu().numpy()
high_scores_idxs = np.where(scores > 0.9)[0].tolist() # Indexes of boxes with scores > 0.7
post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], output[0]['scores'][high_scores_idxs], 0.5).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)
keypoints = []
for kps in output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
  keypoints.append([list(map(int, kp[:2])) for kp in kps])
bboxes = []
for bbox in output[0]['boxes'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
    bboxes.append(list(map(int, bbox.tolist())))

In [ ]:
def visualize_new(image, bboxes, keypoints, image_original=None, bboxes_original=None, keypoints_original=None, scores=[], keypoints_scores=[]):
    fontsize = 1
    i = 0
    for bbox in bboxes:
        start_point = (bbox[0], bbox[1])
        end_point = (bbox[2], bbox[3])
        image = cv2.rectangle(image.copy(), start_point, end_point, (0,255,0), 2)
        if len(scores) > 0:
          image = cv2.putText(image.copy(), "Detected " + str(int(100 * scores[i]) / 100), (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        else:
          image = cv2.putText(image.copy(), "Detected ", (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        i += 1

    j = 0
    for kps in keypoints:
        for idx, kp in enumerate(kps):
            image = cv2.circle(image.copy(), tuple(kp), 5, (255,0,0), 10)
#           image = cv2.putText(image.copy(), " " + keypoints_classes_ids2names[idx], tuple(kp), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1, cv2.LINE_AA)
            if len(keypoints_scores) > 0:
              image = cv2.putText(image.copy(), " " + keypoints_classes_ids2names[idx] + " " + str(int(100 * keypoints_scores[j][idx]) / 100), tuple(kp), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1, cv2.LINE_AA)
            else:
              image = cv2.putText(image.copy(), " " + keypoints_classes_ids2names[idx], tuple(kp), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1, cv2.LINE_AA)
        j += 1
    #print(keypoints)
    #cv2.line(image, tuple(keypoints[0][1]), tuple(keypoints[0][2]), (0,0,255), thickness=2) 
    #cv2.line(image, tuple(keypoints[0][1]), tuple(keypoints[0][2]), (255,0,0), thickness=2) 

    if image_original is None and keypoints_original is None:
        plt.figure(figsize=(40,40))
        plt.imshow(image)

    else:
        for bbox in bboxes_original:
            start_point = (bbox[0], bbox[1])
            end_point = (bbox[2], bbox[3])
            image_original = cv2.rectangle(image_original.copy(), start_point, end_point, (0,255,0), 2)
        
        for kps in keypoints_original:
            for idx, kp in enumerate(kps):
                image_original = cv2.circle(image_original, tuple(kp), 5, (255,0,0), 10)
                image_original = cv2.putText(image_original, " " + keypoints_classes_ids2names[idx], tuple(kp), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 3, cv2.LINE_AA)

        f, ax = plt.subplots(1, 2, figsize=(40, 20))

        ax[0].imshow(image_original)
        ax[0].set_title('Original image', fontsize=fontsize)

        ax[1].imshow(image)
        ax[1].set_title('Transformed image', fontsize=fontsize)


visualize_new(image_show, bboxes, keypoints, scores = scores, keypoints_scores = keypoints_scores)